# Alle notwendigen Inputdaten einlesen

## Imports & Generische Konfig

In [1]:
#Variables
import config

#Preprocessing
import preprocessing_functions as pf

#Libraries
import requests
import json
import pandas as pd
import glob
from bs4 import BeautifulSoup
from sqlalchemy import text
from datetime import date

#Verbindung zur DB aufbauen mit preprocessing function
pf.db_connect(config.db_name)


## Historische Wetterdaten auslesen

In [2]:
#Startdatum + Enddatum definieren: Enddatum = heute, Startdatum = Ab ersten Daten für PM10 History
startdatum = config.date_PM10Start
enddatum = config.date_heute

#Daten aus Open-Meteo (Temperatur, Luftfeuchtigkeit etc.)
response = requests.get('https://api.open-meteo.com/v1/forecast?latitude=47.37&longitude=8.54&hourly=temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,snow_depth,weathercode,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,visibility,evapotranspiration,et0_fao_evapotranspiration,vapor_pressure_deficit,windspeed_10m,windspeed_80m,windspeed_120m,windspeed_180m,winddirection_10m,winddirection_80m,winddirection_120m,winddirection_180m,windgusts_10m,temperature_80m,temperature_120m,temperature_180m,soil_temperature_0cm,soil_temperature_6cm,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_1cm,soil_moisture_1_3cm,soil_moisture_3_9cm,soil_moisture_9_27cm,soil_moisture_27_81cm,uv_index,uv_index_clear_sky,is_day,cape,freezinglevel_height,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,terrestrial_radiation_instant,temperature_1000hPa,temperature_975hPa,temperature_950hPa,temperature_925hPa,temperature_900hPa,temperature_850hPa,temperature_800hPa,temperature_700hPa,temperature_600hPa,temperature_500hPa,temperature_400hPa,temperature_300hPa,temperature_250hPa,temperature_200hPa,temperature_150hPa,temperature_100hPa,temperature_70hPa,temperature_50hPa,temperature_30hPa,relativehumidity_1000hPa,relativehumidity_975hPa,relativehumidity_950hPa,relativehumidity_925hPa,relativehumidity_900hPa,relativehumidity_850hPa,relativehumidity_800hPa,relativehumidity_700hPa,relativehumidity_600hPa,relativehumidity_500hPa,relativehumidity_400hPa,relativehumidity_300hPa,relativehumidity_250hPa,relativehumidity_200hPa,relativehumidity_150hPa,relativehumidity_100hPa,relativehumidity_70hPa,relativehumidity_50hPa,relativehumidity_30hPa,cloudcover_1000hPa,cloudcover_975hPa,cloudcover_950hPa,cloudcover_925hPa,cloudcover_900hPa,cloudcover_850hPa,cloudcover_800hPa,cloudcover_700hPa,cloudcover_600hPa,cloudcover_500hPa,cloudcover_400hPa,cloudcover_300hPa,cloudcover_250hPa,cloudcover_200hPa,cloudcover_150hPa,cloudcover_100hPa,cloudcover_70hPa,cloudcover_50hPa,cloudcover_30hPa,windspeed_1000hPa,windspeed_975hPa,windspeed_950hPa,windspeed_925hPa,windspeed_900hPa,windspeed_850hPa,windspeed_800hPa,windspeed_700hPa,windspeed_600hPa,windspeed_500hPa,windspeed_400hPa,windspeed_300hPa,windspeed_250hPa,windspeed_200hPa,windspeed_150hPa,windspeed_100hPa,windspeed_70hPa,windspeed_50hPa,windspeed_30hPa,winddirection_1000hPa,winddirection_975hPa,winddirection_950hPa,winddirection_925hPa,winddirection_900hPa,winddirection_850hPa,winddirection_800hPa,winddirection_700hPa,winddirection_600hPa,winddirection_500hPa,winddirection_400hPa,winddirection_300hPa,winddirection_250hPa,winddirection_200hPa,winddirection_150hPa,winddirection_100hPa,winddirection_70hPa,winddirection_50hPa,winddirection_30hPa&models=best_match,ecmwf_ifs04,metno_nordic,gfs_seamless,jma_seamless,icon_seamless,gem_seamless,meteofrance_seamless&daily=weathercode,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,uv_index_max,uv_index_clear_sky_max,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&start_date=' + startdatum + '&end_date=' + enddatum + '&timezone=Europe%2FZurich')
weather_data = json.loads(response.content)
daily_data = weather_data['daily']

# DataFrame erstellen:
weather_df = pd.DataFrame(daily_data)

# Datum als Idex
weather_df.set_index('time', inplace=True)

#Daten bereinigen (Entfernung NAs)
weather_df=weather_df.drop(columns=['shortwave_radiation_sum_jma_seamless', 'et0_fao_evapotranspiration_jma_seamless'])

#Datum aus der Spalte sunrise gewinnen, in einer Spalte Datum speichern und umformatieren
weather_df['Datum'] = weather_df['sunrise'].str.slice(stop=10)
weather_df['Datum'] = weather_df['Datum'].apply(pf.format_date)

#Daten in Wetter-Table der DB schreiben mit preprocessing Funktion
weather_df.to_sql(config.db_weather, config.db_login, if_exists='append', index=False)


136

## Historische Feinstaub Daten für Zürich auslesen

In [3]:
url = 'https://data.stadt-zuerich.ch/dataset/ugz_luftschadstoffmessung_tageswerte'

# Verbindung zur Webseite herstellen
response = requests.get(url)

# Webseite parsen
soup = BeautifulSoup(response.text, 'html.parser')

# Alle CSV-Links finden
csv_links = []
for a in soup.find_all('a', href=True):
    link = a['href']
    if link.endswith('.csv'):
        csv_links.append(link)

# CSV-Dateien herunterladen
for link in csv_links:
    response = requests.get(link)
    with open('csv-files/'+link.split('/')[-1], 'wb') as f:
        f.write(response.content)

# Liste aller ugz CSV-Dateien im Verzeichnis csv-files
csv_files = glob.glob('csv-files/*.csv')

# Leeres DataFrame zum Speichern der gefilterten Daten erstellen
AQI_history_ZH = pd.DataFrame()
    
# Durch alle CSV-Dateien iterieren und Daten filtern
for file in csv_files:
    df = pd.read_csv(file)
    df_filtered = df[df['Parameter'] == 'PM10']
    df_filtered = df_filtered[df_filtered['Standort'] == 'Zch_Stampfenbachstrasse']
    df_filtered['Datum'] = df_filtered['Datum'].apply(pf.format_timestamp)
    AQI_history_ZH = AQI_history_ZH.append(df_filtered)

# Index neu setzen
AQI_history_ZH.reset_index(drop=True, inplace=True)

# Dataframe in die DB schreiben
AQI_history_ZH.to_sql(config.db_AQI_history, config.db_login, if_exists='append', index=False)


164

## Aktuelle Feinstaubdaten für verschiedene Städte auslesen (30 Tage)

In [4]:

#Daten aus Air Quality Index laden
from bs4 import BeautifulSoup

URL = "https://www.bafu.admin.ch/bafu/de/home/themen/luft/zustand/daten/luftbelastung--historische-daten/nabel--tageswerte-der-letzten-30--tage/werte-der-letzten-30-tage-nabel--feinstaub--pm10-.html"

f = requests.get(URL)
soup = BeautifulSoup(f.text)

#slicer - Unnötige Informationen entfernen: Alle Wörter vor dem string "Hoch-geb." werden entfernt sowie alle Wörter nach "Legende". Genau in dieser Spanne sind die eigentlichen Daten.
strValue = soup.get_text()

bereinigteListe = strValue.split("Hoch-geb.\n\n\n", 1)
if len(bereinigteListe) > 0:
    strValue=bereinigteListe[1]

strValue = strValue.split("\n\n\n\n\n\n\n     Legende", 1)[0]

#Daten aus Air Quality Index in Tabellen-Format bringen
from io import StringIO

data = strValue.split('\n\n')

data[0] = '\n\n' + data[0]

columns = data[0].split('\n')[1:]

rows = [row[1:] for row in data[1:]]

rows = [row.split('\n') for row in data[1:]]

rows = [row[:len(columns)] for row in rows]

AQI_df = pd.DataFrame(rows, columns=columns)

AQI_df = AQI_df.drop('', axis=1)

#Ganze Table löschen, fals mal wieder was verschoben ist...
#pf.db_drop_table(config.db_name, config.db_AQI_RAW)

#Daten in AQI-Table der DB schreiben mit preprocessing Funktion
AQI_df.to_sql(config.db_AQI_RAW, config.db_login, if_exists='append', index=False)


#Die überflüssigen Daten aus dem AQI DF löschen (Air Quality-Spalten für die anderen Kantone sowie die Datumsspalte) - Hierzu Hinweis: Die Spalten sind irgendwie verrutscht, deshalb muss die Spalte "BAS" behalten werden, sie zeigt aber die Daten für Zürich an.
AQI_df = AQI_df.iloc[1:]

AQI_df = AQI_df[['BAS']]

print(AQI_df)

#--------------------------------------------------------------------

#Umbenennen der Spalte in "AQI-BAFU" (Air Quality Index - Bundesamt für Umwelt)
AQI_df = AQI_df.rename(columns={"BAS": "AQI - BAFU"})

#Index zurücksetzen, damit die beiden Tabellen verbunden werden können:
AQI_df_reset = AQI_df.reset_index(drop=True)
weather_df_reset = weather_df.reset_index(drop=True)

#Tabellen verbinden
all_df = pd.concat([weather_df_reset, AQI_df_reset], axis=1)

#print(all_df)
all_df.to_csv(config.csv_export, index=False)



   BAS
1    8
2   11
3   13
4   13
5    6
6   10
7    8
8    4
9    5
10   4
11   9
12   9
13   8
14   9
15  10
16   8
17  10
18  13
19  13
20  12
21  11
22   8
23   6
24   9
25   4
26   4
27   6
28   9
29   9
30   8


## DB Testen für eingelesene Daten

In [5]:
# Check Output von DB für weather
SQLquery = text('SELECT * FROM ' + config.db_weather + ' AS weather')
df_METEODB = pd.read_sql(SQLquery, con=config.db_login.connect())

df_METEODB.head()

,weathercode_best_match,temperature_2m_max_best_match,temperature_2m_min_best_match,apparent_temperature_max_best_match,apparent_temperature_min_best_match,sunrise,sunset,uv_index_max_best_match,uv_index_clear_sky_max_best_match,precipitation_sum_best_match,...,apparent_temperature_min_meteofrance_seamless,precipitation_sum_meteofrance_seamless,snowfall_sum_meteofrance_seamless,precipitation_hours_meteofrance_seamless,windspeed_10m_max_meteofrance_seamless,windgusts_10m_max_meteofrance_seamless,winddirection_10m_dominant_meteofrance_seamless,shortwave_radiation_sum_meteofrance_seamless,et0_fao_evapotranspiration_meteofrance_seamless,Datum
0,80.0,9.0,7.5,7.3,5.0,2023-04-16T06:34,2023-04-16T20:17,1.25,5.30,20.0,...,4.9,14.2,0.0,19.0,9.5,25.2,250.0,6.38,0.99,16.04.23
1,80.0,13.9,7.9,10.0,6.7,2023-04-17T06:32,2023-04-17T20:18,4.60,5.60,2.0,...,5.6,4.1,0.0,12.0,18.0,39.2,31.0,20.55,2.92,17.04.23
2,3.0,16.3,8.1,13.6,6.1,2023-04-18T06:30,2023-04-18T20:19,5.15,5.70,0.0,...,5.1,0.0,0.0,0.0,17.3,38.5,39.0,23.10,3.53,18.04.23
3,80.0,12.8,7.0,10.6,2.8,2023-04-19T06:28,2023-04-19T20:21,3.00,5.65,0.2,...,4.4,1.3,0.0,3.0,18.1,42.1,21.0,18.17,2.76,19.04.23
4,61.0,7.3,5.5,5.5,1.8,2023-04-20T06:26,2023-04-20T20:22,0.75,5.40,4.7,...,1.8,1.3,0.0,8.0,11.6,32.0,22.0,11.07,1.38,20.04.23


In [6]:
# Check Output von DB für AQI_history
SQLquery = text('SELECT * FROM ' + config.db_AQI_history + ' AS AQI_History')
df_AQIHistoryDB = pd.read_sql(SQLquery, con=config.db_login.connect())

df_AQIHistoryDB.head()

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
0,04.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,22.84,bereinigt
1,05.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,22.63,bereinigt
2,06.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,10.93,bereinigt
3,07.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,NaN,bereinigt
4,08.01.01,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,18.01,bereinigt


In [7]:
# Check Output von DB für AQI_RAW
SQLquery = text('SELECT * FROM ' + config.db_AQI_RAW + ' AQI')
df_AQIDB = pd.read_sql(SQLquery, con=config.db_login.connect())

df_AQIDB.head()

,Datum,BER,LAU,LUG,ZUE,BAS,DUE,HAE,SIO,MAG,PAY,TAE,BRM,CHA,RIG,DAV,JUN
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,16.04.23,9,8,2,4,8,5,8,5,4,6,3,4,5,2,0,0
2,17.04.23,7,5,6,5,11,5,7,8,7,5,5,3,3,2,1,0
3,18.04.23,12,10,8,9,13,9,10,11,10,9,9,8,6,8,3,1
4,19.04.23,16,15,15,11,13,9,10,19,14,12,10,9,8,9,5,2


In [9]:
#Gemeinsame Abfrage der Tabellen
SQLquery = text('SELECT * FROM ' + config.db_weather + ' AS w JOIN ' + config.db_AQI_history + ' AS aqi ON w."Datum" = aqi."Datum"')
df_METEODB = pd.read_sql(SQLquery, con=config.db_login.connect())

df_METEODB.head()

,weathercode_best_match,temperature_2m_max_best_match,temperature_2m_min_best_match,apparent_temperature_max_best_match,apparent_temperature_min_best_match,sunrise,sunset,uv_index_max_best_match,uv_index_clear_sky_max_best_match,precipitation_sum_best_match,...,shortwave_radiation_sum_meteofrance_seamless,et0_fao_evapotranspiration_meteofrance_seamless,Datum,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
2743,80.0,15.2,10.7,14.4,9.5,2023-05-14T05:48,2023-05-14T20:55,5.15,6.3,3.8,...,12.42,1.91,14.05.23,14.05.23,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,8.66,provisorisch
2744,80.0,16.0,11.2,15.2,10.5,2023-05-15T05:47,2023-05-15T20:56,6.65,6.7,2.1,...,19.10,2.97,15.05.23,15.05.23,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,7.04,provisorisch
2745,80.0,16.0,11.2,15.2,10.5,2023-05-15T05:47,2023-05-15T20:56,6.65,6.7,2.1,...,19.10,2.97,15.05.23,15.05.23,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,7.04,provisorisch
2746,80.0,16.0,11.2,15.2,10.5,2023-05-15T05:47,2023-05-15T20:56,6.65,6.7,2.1,...,19.10,2.97,15.05.23,15.05.23,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,7.04,provisorisch
2747,80.0,16.0,11.2,15.2,10.5,2023-05-15T05:47,2023-05-15T20:56,6.65,6.7,2.1,...,19.10,2.97,15.05.23,15.05.23,Zch_Stampfenbachstrasse,PM10,d1,µg/m3,7.04,provisorisch
